In [2]:
import xlwings

In [3]:
xlwings.sheets

Sheets([<Sheet [log.xlsx]Sheet1>])

In [4]:
xlwings.Range("A1")

<Range [log.xlsx]Sheet1!$A$1>

In [6]:
xlwings.Range("A1").expand()

<Range [log.xlsx]Sheet1!$A$1:$B$10>

In [9]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번쨰 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [10]:
column_names = total_rows[0]
column_names

['message', 'browser']

In [13]:
for row in total_rows[1:]:
    row_dict = dict(zip(column_names, row))
    print(row_dict)

{'message': '첫 로그', 'browser': 'chrome'}
{'message': '두번째 로그', 'browser': 'chrome'}
{'message': '세번쨰 로그', 'browser': 'chrome'}
{'message': '로그#4', 'browser': 'chrome'}
{'message': '로그#5', 'browser': 'chrome'}
{'message': '로그#6', 'browser': 'chrome'}
{'message': '로그#7', 'browser': 'chrome'}
{'message': '로그#8', 'browser': 'chrome'}
{'message': '로그#9', 'browser': 'chrome'}


In [15]:
# list comprehension
rows_data = [
    dict(zip(column_names, row))
    for row in total_rows[1:]
]
rows_data

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번쨰 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 정리

In [17]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번쨰 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 데이터베이스 접속

In [23]:
DATABASE = {
        "host":'127.0.0.1',
        "user":'root', 
        "passwd":'1234',  
        "db":'myproj11_db', 
        "charset":'utf8'
}

In [24]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log;")
print(cursor.fetchall())
cursor.close()

()


# with 절

In [30]:
from contextlib import contextmanager

@contextmanager
def db_cursor():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()

In [31]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    print(cursor.fetchall())

()


In [42]:
column_data = {'message': '첫 로그', 'browser': 'chrome'}

with db_cursor() as cursor:
    for row_dict in get_rows_data():
        sql = "INSERT INTO LOG(message, browser) VALUES(%(message)s, %(browser)s)"
        cursor.execute(sql, row_dict)

In [48]:
with db_cursor() as cursor:
    sql = "SELECT * FROM LOG"
    cursor.execute(sql)
#     while True:
#         row = cursor.fetchone()
#         print(row)
#         if row is None:
#             break
#         print(row)
    for row in cursor:
        print(row)

(13, '', '')
(14, '', '')
(15, '', '')


# SQL Injection 공격의 예

In [57]:
with db_cursor() as cursor:
    log_id = "1 or 1"
#     sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
#     sql = "SELECT * FROM LOG WHERE id = 13 or 1"
    sql = "SELECT * FROM log WHERE id = %s"
    cursor.execute(sql, [log_id])
    for row in cursor:
        print(row)

In [32]:
get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번쨰 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

# 데이터 매핑의 예

In [60]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
#     for row in cursor:
#         row[0] #id
#         row[1] #message
#         row[2] # chrome
#         print(row)
rows_data

((13, 'message', 'chrome'),
 (14, 'message', 'chrome'),
 (15, 'message', 'chrome'))

# dict으로의 변환

## namedtuple

In [62]:
from collections import namedtuple

Log = namedtuple('Log', 'id message browser')

In [65]:
log = (1, 'message', 'chrome')
print(log[0])
print(log[1])
print(log[2])


1
message
chrome


In [66]:
log = Log(1, 'message', 'chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [69]:
log.id, log.message, log.browser

(1, 'message', 'chrome')

In [70]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for row in cursor.fetchall()]

rows_data

[Log(id=13, message='message', browser='chrome'),
 Log(id=14, message='message', browser='chrome'),
 Log(id=15, message='message', browser='chrome')]